In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import keras as keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from keras import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from copy import deepcopy

Using TensorFlow backend.


In [2]:
red = pd.read_csv('data/winequality-red1.csv')
white = pd.read_csv('data/winequality-white1.csv')

In [3]:
red['quality'].replace(to_replace=[0,1,2,3,4,5], value=1, inplace=True)
red['quality'].replace(to_replace=[6], value=2, inplace=True)
red['quality'].replace(to_replace=[7,8,9,10], value=3, inplace=True)
X_red = red[
['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']]
Y_red = red[['quality']]
white['quality'].replace(to_replace=[0,1,2,3,4,5], value=1, inplace=True)
white['quality'].replace(to_replace=[6], value=2, inplace=True)
white['quality'].replace(to_replace=[7,8,9,10], value=3, inplace=True)
X_white = white[
['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']]

Y_white = white[['quality']]
X_red = X_red.values
Y_red = Y_red.values
X_white = X_white.values
Y_white = Y_white.values
number_of_features = 11

In [4]:
def rsme(targets, outputs):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(targets, outputs))))  

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Create function returning a compiled network
def create_network():
    network = models.Sequential()
    #network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    #network.add(layers.Dense(units=9, activation = 'relu'))
    #network.add(layers.Dense(units=6, activation = 'relu'))
    #network.add(layers.Dense(units=18, activation = 'relu'))
    network.add(layers.Dense(3, activation='softmax'))

    #network.compile(loss='binary_crossentropy', # Cross-entropy
   #                 optimizer='rmsprop', # Root Mean Square Propagation
    #                metrics=['accuracy']) # Accuracy performance metric
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    
    # Return compiled network
    return network


neural_network = KerasClassifier(build_fn=create_network, 
                                     epochs=10, 
                                     batch_size=100, 
                                     verbose=0)

## Start strategically exploration

In [5]:
def create_network1():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='linear'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network1, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Linear, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

One Layer Linear, Softmax
Red
              precision    recall  f1-score   support

           1       0.69      0.76      0.72       744
           2       0.52      0.56      0.54       638
           3       0.49      0.21      0.30       217

   micro avg       0.61      0.61      0.61      1599
   macro avg       0.57      0.51      0.52      1599
weighted avg       0.60      0.61      0.59      1599

White
              precision    recall  f1-score   support

           1       0.59      0.46      0.52      1640
           2       0.50      0.72      0.59      2198
           3       0.55      0.23      0.32      1060

   micro avg       0.53      0.53      0.53      4898
   macro avg       0.55      0.47      0.48      4898
weighted avg       0.54      0.53      0.51      4898

RSME red
0.9342910510695603
RSME white
0.9284640135705642


In [6]:
def create_network2():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='sigmoid'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network2, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Sigmoid, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

One Layer Sigmoid, Softmax
Red
              precision    recall  f1-score   support

           1       0.68      0.78      0.73       744
           2       0.54      0.56      0.55       638
           3       0.59      0.22      0.32       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.60      0.52      0.53      1599
weighted avg       0.61      0.62      0.60      1599

White
              precision    recall  f1-score   support

           1       0.62      0.59      0.60      1640
           2       0.53      0.65      0.58      2198
           3       0.55      0.35      0.42      1060

   micro avg       0.56      0.56      0.56      4898
   macro avg       0.57      0.53      0.54      4898
weighted avg       0.57      0.56      0.56      4898

RSME red
0.9315169181939235
RSME white
0.9790891361083127


In [7]:
def create_network3():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network3, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

One Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.68      0.77      0.73       744
           2       0.54      0.55      0.55       638
           3       0.57      0.30      0.40       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.60      0.54      0.56      1599
weighted avg       0.61      0.62      0.61      1599

White
              precision    recall  f1-score   support

           1       0.59      0.66      0.62      1640
           2       0.53      0.54      0.54      2198
           3       0.49      0.38      0.43      1060

   micro avg       0.55      0.55      0.55      4898
   macro avg       0.54      0.53      0.53      4898
weighted avg       0.54      0.55      0.54      4898

RSME red
0.9481313345897879
RSME white
1.0221389663014981


In [8]:
def create_network4():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='selu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network4, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Selu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

One Layer Selu, Softmax
Red
              precision    recall  f1-score   support

           1       0.69      0.75      0.72       744
           2       0.54      0.56      0.55       638
           3       0.57      0.33      0.42       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.60      0.55      0.56      1599
weighted avg       0.61      0.62      0.61      1599

White
              precision    recall  f1-score   support

           1       0.61      0.60      0.60      1640
           2       0.52      0.61      0.56      2198
           3       0.48      0.34      0.40      1060

   micro avg       0.55      0.55      0.55      4898
   macro avg       0.54      0.51      0.52      4898
weighted avg       0.54      0.55      0.54      4898

RSME red
0.9513496388698337
RSME white
0.9938300901140824


In [9]:
def create_network5():
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='tanh'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network5, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("One Layer Tanh, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

One Layer Tanh, Softmax
Red
              precision    recall  f1-score   support

           1       0.68      0.76      0.72       744
           2       0.54      0.55      0.54       638
           3       0.56      0.30      0.39       217

   micro avg       0.61      0.61      0.61      1599
   macro avg       0.59      0.54      0.55      1599
weighted avg       0.61      0.61      0.60      1599

White
              precision    recall  f1-score   support

           1       0.61      0.61      0.61      1640
           2       0.52      0.64      0.57      2198
           3       0.56      0.31      0.40      1060

   micro avg       0.56      0.56      0.56      4898
   macro avg       0.56      0.52      0.53      4898
weighted avg       0.56      0.56      0.55      4898

RSME red
0.9486486873459298
RSME white
0.9770046005211256


In [10]:
def create_network6():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network6, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Linear, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Linear, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.77      0.74       744
           2       0.55      0.57      0.56       638
           3       0.54      0.34      0.41       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.60      0.56      0.57      1599
weighted avg       0.62      0.63      0.62      1599

White
              precision    recall  f1-score   support

           1       0.63      0.59      0.61      1640
           2       0.54      0.67      0.60      2198
           3       0.58      0.34      0.43      1060

   micro avg       0.57      0.57      0.57      4898
   macro avg       0.58      0.53      0.54      4898
weighted avg       0.58      0.57      0.56      4898

RSME red
0.956215685792089
RSME white
0.9719457961596918


In [11]:
def create_network7():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='sigmoid'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network7, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Sigmoid, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Sigmoid, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.78      0.74       744
           2       0.56      0.57      0.56       638
           3       0.55      0.35      0.43       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.57      0.58      1599
weighted avg       0.63      0.64      0.63      1599

White
              precision    recall  f1-score   support

           1       0.67      0.62      0.64      1640
           2       0.55      0.68      0.61      2198
           3       0.59      0.39      0.47      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.56      0.57      4898
weighted avg       0.60      0.59      0.59      4898

RSME red
0.9587317497144066
RSME white
0.9823514365552873


In [12]:
def create_network8():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network8, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.78      0.75       744
           2       0.58      0.59      0.58       638
           3       0.60      0.42      0.50       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.60      0.61      1599
weighted avg       0.65      0.65      0.65      1599

White
              precision    recall  f1-score   support

           1       0.68      0.64      0.66      1640
           2       0.57      0.65      0.61      2198
           3       0.59      0.48      0.53      1060

   micro avg       0.61      0.61      0.61      4898
   macro avg       0.61      0.59      0.60      4898
weighted avg       0.61      0.61      0.61      4898

RSME red
0.9651598750207818
RSME white
1.0060669873081927


In [13]:
def create_network9():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='selu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network9, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Selu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Selu, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.76      0.74       744
           2       0.57      0.59      0.58       638
           3       0.57      0.40      0.47       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.62      0.58      0.60      1599
weighted avg       0.64      0.64      0.64      1599

White
              precision    recall  f1-score   support

           1       0.67      0.63      0.65      1640
           2       0.56      0.66      0.61      2198
           3       0.60      0.43      0.50      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.61      0.60      0.60      4898

RSME red
0.9617745049078577
RSME white
0.9927238384016788


In [14]:
def create_network10():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='tanh'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network10, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Tanh, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Tanh, Softmax
Red
              precision    recall  f1-score   support

           1       0.70      0.78      0.74       744
           2       0.56      0.57      0.57       638
           3       0.60      0.37      0.46       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.62      0.57      0.59      1599
weighted avg       0.63      0.64      0.63      1599

White
              precision    recall  f1-score   support

           1       0.66      0.65      0.66      1640
           2       0.57      0.64      0.60      2198
           3       0.58      0.45      0.51      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.60      0.58      0.59      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.9568022486768938
RSME white
1.0048220150694551


## Including Feature Selection

In [15]:
## selection from naive bayes notebook
X_red_sel = red[
[#'fixed acidity', -> low importances, thus removed
 'volatile acidity', 
 #'citric acid', -> low importances, thus removed
 #'residual sugar', -> low importances, thus removed
 #'chlorides', -> low importances, thus removed
 #'free sulfur dioxide', -> low importances, thus removed
 'total sulfur dioxide',
 'density',
 #'pH', -> low importances, thus removed
 'sulphates',
 'alcohol']]
Y_red_sel = red[['quality']]
X_white_sel = white[
['fixed acidity',
 'volatile acidity',
 #'citric acid', -> low importances, thus removed
 'residual sugar',
 #'chlorides', -> low importances, thus removed
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 #'pH', -> low importances, thus removed
 #'sulphates', -> low importances, thus removed
 'alcohol']]

Y_white_sel = white[['quality']]
X_red_sel = X_red_sel.values
Y_red_sel = Y_red_sel.values
X_white_sel = X_white_sel.values
Y_white_sel = Y_white_sel.values

In [16]:
neural_network = KerasClassifier(build_fn=create_network6, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Linaer, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red_sel, Y_red_sel, cv=cv) 
print(classification_report(Y_red_sel, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white_sel, Y_white_sel, cv=cv) 
print(classification_report(Y_white_sel, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red_sel-prediction_red)*(Y_red_sel-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white_sel-prediction_white)*(Y_white_sel-prediction_white))))

Batch Normalize Layer, one Layer Linaer, Softmax
Red
              precision    recall  f1-score   support

           1       0.69      0.77      0.73       744
           2       0.54      0.55      0.54       638
           3       0.57      0.33      0.42       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.60      0.55      0.56      1599
weighted avg       0.61      0.62      0.61      1599

White
              precision    recall  f1-score   support

           1       0.64      0.57      0.61      1640
           2       0.52      0.67      0.59      2198
           3       0.54      0.31      0.39      1060

   micro avg       0.56      0.56      0.56      4898
   macro avg       0.57      0.52      0.53      4898
weighted avg       0.57      0.56      0.55      4898

RSME red
0.9519244090298621
RSME white
0.9662519779437131


In [17]:
neural_network = KerasClassifier(build_fn=create_network7, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Sigmoid, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red_sel, Y_red_sel, cv=cv) 
print(classification_report(Y_red_sel, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white_sel, Y_white_sel, cv=cv) 
print(classification_report(Y_white_sel, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red_sel-prediction_red)*(Y_red_sel-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white_sel-prediction_white)*(Y_white_sel-prediction_white))))

Batch Normalize Layer, one Layer Sigmoid, Softmax
Red
              precision    recall  f1-score   support

           1       0.70      0.78      0.74       744
           2       0.55      0.57      0.56       638
           3       0.60      0.32      0.42       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.62      0.56      0.57      1599
weighted avg       0.63      0.63      0.62      1599

White
              precision    recall  f1-score   support

           1       0.63      0.65      0.64      1640
           2       0.55      0.62      0.58      2198
           3       0.57      0.39      0.46      1060

   micro avg       0.58      0.58      0.58      4898
   macro avg       0.58      0.55      0.56      4898
weighted avg       0.58      0.58      0.58      4898

RSME red
0.9477643358391514
RSME white
0.9991625646566592


In [18]:
neural_network = KerasClassifier(build_fn=create_network8, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red_sel, Y_red_sel, cv=cv) 
print(classification_report(Y_red_sel, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white_sel, Y_white_sel, cv=cv) 
print(classification_report(Y_white_sel, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red_sel-prediction_red)*(Y_red_sel-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white_sel-prediction_white)*(Y_white_sel-prediction_white))))

Batch Normalize Layer, one Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.77      0.74       744
           2       0.57      0.58      0.57       638
           3       0.62      0.41      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.63      0.59      0.60      1599
weighted avg       0.64      0.64      0.64      1599

White
              precision    recall  f1-score   support

           1       0.67      0.64      0.66      1640
           2       0.56      0.66      0.61      2198
           3       0.60      0.41      0.48      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.57      0.58      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.959784088758095
RSME white
0.9901362622717693


In [19]:
neural_network = KerasClassifier(build_fn=create_network9, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Selu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red_sel, Y_red_sel, cv=cv) 
print(classification_report(Y_red_sel, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white_sel, Y_white_sel, cv=cv) 
print(classification_report(Y_white_sel, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red_sel-prediction_red)*(Y_red_sel-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white_sel-prediction_white)*(Y_white_sel-prediction_white))))

Batch Normalize Layer, one Layer Selu, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.77      0.74       744
           2       0.54      0.57      0.56       638
           3       0.60      0.34      0.43       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.62      0.56      0.58      1599
weighted avg       0.63      0.63      0.62      1599

White
              precision    recall  f1-score   support

           1       0.65      0.63      0.64      1640
           2       0.55      0.65      0.60      2198
           3       0.59      0.39      0.47      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.56      0.57      4898
weighted avg       0.59      0.59      0.58      4898

RSME red
0.9496825473532942
RSME white
0.9880966322140148


In [20]:
neural_network = KerasClassifier(build_fn=create_network10, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Tanh, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red_sel, Y_red_sel, cv=cv) 
print(classification_report(Y_red_sel, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white_sel, Y_white_sel, cv=cv) 
print(classification_report(Y_white_sel, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red_sel-prediction_red)*(Y_red_sel-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white_sel-prediction_white)*(Y_white_sel-prediction_white))))

Batch Normalize Layer, one Layer Tanh, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.77      0.74       744
           2       0.56      0.57      0.56       638
           3       0.63      0.40      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.63      0.58      0.60      1599
weighted avg       0.64      0.64      0.64      1599

White
              precision    recall  f1-score   support

           1       0.66      0.64      0.65      1640
           2       0.56      0.66      0.61      2198
           3       0.59      0.42      0.49      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.57      0.58      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.9568842039048707
RSME white
0.9914566458908961


## experimental adding of layers

In [21]:
def create_network11():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='relu'))
    network.add(layers.Dense(units=9, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network11, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, two Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, two Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.73      0.78      0.75       744
           2       0.60      0.59      0.59       638
           3       0.60      0.47      0.53       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.64      0.61      0.62      1599
weighted avg       0.66      0.66      0.66      1599

White
              precision    recall  f1-score   support

           1       0.66      0.67      0.67      1640
           2       0.59      0.63      0.61      2198
           3       0.60      0.50      0.55      1060

   micro avg       0.62      0.62      0.62      4898
   macro avg       0.62      0.60      0.61      4898
weighted avg       0.62      0.62      0.62      4898

RSME red
0.9749917864515196
RSME white
1.015956911168021


In [22]:
def create_network12():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    network.add(layers.Dense(units=9, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network12, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Linear, one Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Linear, one Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.77      0.75       744
           2       0.57      0.58      0.58       638
           3       0.58      0.40      0.47       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.62      0.59      0.60      1599
weighted avg       0.64      0.65      0.64      1599

White
              precision    recall  f1-score   support

           1       0.66      0.62      0.64      1640
           2       0.55      0.66      0.60      2198
           3       0.60      0.40      0.48      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.56      0.57      4898
weighted avg       0.60      0.59      0.59      4898

RSME red
0.9633246576945603
RSME white
0.9854449226083525


In [5]:
def create_network13():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='relu'))
    network.add(layers.Dense(units=9, activation='linear'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network13, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Relu, one Layer Linear, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Relu, one Layer Linear, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.78      0.75       744
           2       0.59      0.59      0.59       638
           3       0.61      0.46      0.52       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.64      0.61      0.62      1599
weighted avg       0.66      0.66      0.66      1599

White
              precision    recall  f1-score   support

           1       0.69      0.62      0.65      1640
           2       0.58      0.64      0.61      2198
           3       0.58      0.53      0.56      1060

   micro avg       0.61      0.61      0.61      4898
   macro avg       0.61      0.60      0.60      4898
weighted avg       0.61      0.61      0.61      4898

RSME red
0.9709450548918501
RSME white
1.0128847186091983


In [6]:
def create_network14():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='sigmoid'))
    network.add(layers.Dense(units=9, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network14, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Sigmoid, one Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Sigmoid, one Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.77      0.74       744
           2       0.56      0.58      0.57       638
           3       0.56      0.37      0.45       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.57      0.59      1599
weighted avg       0.63      0.64      0.63      1599

White
              precision    recall  f1-score   support

           1       0.66      0.63      0.65      1640
           2       0.56      0.65      0.60      2198
           3       0.60      0.46      0.52      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.9586876902009007
RSME white
1.0004278712435513


In [7]:
def create_network15():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    network.add(layers.Dense(units=9, activation='sigmoid'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network15, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Linear, one Layer Sigmoid, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Linear, one Layer Sigmoid, Softmax
Red
              precision    recall  f1-score   support

           1       0.70      0.77      0.73       744
           2       0.56      0.54      0.55       638
           3       0.57      0.41      0.48       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.63      0.63      1599

White
              precision    recall  f1-score   support

           1       0.65      0.62      0.64      1640
           2       0.55      0.65      0.60      2198
           3       0.58      0.40      0.47      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.56      0.57      4898
weighted avg       0.59      0.59      0.58      4898

RSME red
0.970043735025706
RSME white
0.9893273864986711


In [8]:
def create_network16():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='selu'))
    network.add(layers.Dense(units=9, activation='relu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network16, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, one Layer Selu, one Layer Relu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, one Layer Selu, one Layer Relu, Softmax
Red
              precision    recall  f1-score   support

           1       0.72      0.78      0.75       744
           2       0.58      0.60      0.59       638
           3       0.61      0.40      0.48       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.64      0.59      0.61      1599
weighted avg       0.65      0.65      0.65      1599

White
              precision    recall  f1-score   support

           1       0.67      0.64      0.65      1640
           2       0.57      0.64      0.60      2198
           3       0.59      0.47      0.53      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.61      0.60      0.60      4898

RSME red
0.9582575956286976
RSME white
1.0042817572651357


In [9]:
def create_network17():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='selu'))
    network.add(layers.Dense(units=9, activation='selu'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network17, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, two Layer Selu, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, two Layer Selu, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.80      0.75       744
           2       0.59      0.57      0.58       638
           3       0.57      0.41      0.48       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.59      0.60      1599
weighted avg       0.65      0.65      0.65      1599

White
              precision    recall  f1-score   support

           1       0.66      0.65      0.66      1640
           2       0.57      0.66      0.61      2198
           3       0.61      0.45      0.52      1060

   micro avg       0.61      0.61      0.61      4898
   macro avg       0.62      0.59      0.60      4898
weighted avg       0.61      0.61      0.61      4898

RSME red
0.9687825373962528
RSME white
0.9990252603979141


In [10]:
def create_network18():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='linear'))
    network.add(layers.Dense(units=9, activation='linear'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network18, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, two Layer Linear, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, two Layer Linear, Softmax
Red
              precision    recall  f1-score   support

           1       0.70      0.77      0.74       744
           2       0.55      0.56      0.55       638
           3       0.54      0.31      0.40       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.59      0.55      0.56      1599
weighted avg       0.62      0.62      0.62      1599

White
              precision    recall  f1-score   support

           1       0.64      0.58      0.61      1640
           2       0.54      0.67      0.60      2198
           3       0.57      0.35      0.43      1060

   micro avg       0.57      0.57      0.57      4898
   macro avg       0.58      0.53      0.55      4898
weighted avg       0.58      0.57      0.57      4898

RSME red
0.9526634761839756
RSME white
0.9733073887633493


In [11]:
def create_network19():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='sigmoid'))
    network.add(layers.Dense(units=9, activation='sigmoid'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network19, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, two Layer Sigmoid, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, two Layer Sigmoid, Softmax
Red
              precision    recall  f1-score   support

           1       0.70      0.78      0.74       744
           2       0.56      0.54      0.55       638
           3       0.56      0.40      0.46       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.61      0.57      0.58      1599
weighted avg       0.62      0.63      0.62      1599

White
              precision    recall  f1-score   support

           1       0.66      0.64      0.65      1640
           2       0.56      0.67      0.61      2198
           3       0.61      0.38      0.47      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.57      0.58      4898
weighted avg       0.61      0.60      0.60      4898

RSME red
0.9685770237710666
RSME white
0.9831579537684825


In [12]:
def create_network20():
    network = models.Sequential()
    network.add(layers.BatchNormalization())
    network.add(layers.Dense(units=64, activation='tanh'))
    network.add(layers.Dense(units=9, activation='tanh'))
    network.add(layers.Dense(3, activation='softmax'))
    
    network.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', rsme])
    return network

neural_network = KerasClassifier(build_fn=create_network20, 
                                     epochs=100, 
                                     batch_size=100, 
                                     verbose=0)
print("Batch Normalize Layer, two Layer Tanh, Softmax")
print("Red")
prediction_red = cross_val_predict(neural_network, X_red, Y_red, cv=cv) 
print(classification_report(Y_red, prediction_red))
print("White")
prediction_white = cross_val_predict(neural_network, X_white, Y_white, cv=cv) 
print(classification_report(Y_white, prediction_white))
print("RSME red")
print(sqrt(np.mean((Y_red-prediction_red)*(Y_red-prediction_red))))
print('RSME white')
print(sqrt(np.mean((Y_white-prediction_white)*(Y_white-prediction_white))))

Batch Normalize Layer, two Layer Tanh, Softmax
Red
              precision    recall  f1-score   support

           1       0.71      0.78      0.74       744
           2       0.58      0.55      0.56       638
           3       0.58      0.44      0.50       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.62      0.59      0.60      1599
weighted avg       0.64      0.64      0.64      1599

White
              precision    recall  f1-score   support

           1       0.67      0.64      0.66      1640
           2       0.57      0.63      0.60      2198
           3       0.58      0.49      0.53      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.59      0.59      4898
weighted avg       0.60      0.60      0.60      4898

RSME red
0.9736709218873512
RSME white
1.0120039888610664
